In [1]:
import pandas as pd
from datetime import datetime
import os

path = "./timestamp_creator"

In [2]:
# Only should be one report in json format in this folder
input_data = os.listdir(path+"/input")[0]
input_data

'complete_500401b5-622f-48ce-8cc5-5a6d48c80391.json'

In [3]:
sessions = pd.read_json(path+"/input/"+input_data)
sessions = sessions.query('consent == True & close_date != None')
sessions

,_id,close_date,consent,creation_date,current_step,responses,settings,test_id,user
0,GQOhQlQTL9rrtn0IVdMsf,"Thu, 28 Oct 2021 13:47:52 GMT",True,"Thu, 28 Oct 2021 13:15:03 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1635423686, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '30', 'color': {'name': 'blue', 'value..."
1,lvL_pTKXF0QLQiH3jsXJH,"Thu, 04 Nov 2021 09:44:47 GMT",True,"Thu, 04 Nov 2021 09:03:04 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636017250, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '38', 'color': {'name': 'blue', 'value..."
2,OJ6xmryicQnoNVqh6f2EX,"Thu, 04 Nov 2021 11:27:04 GMT",True,"Thu, 04 Nov 2021 10:51:25 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636023608, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '23', 'color': {'name': 'blue', 'value..."
3,bZqZXJl9xdI1bgJIkefIm,"Thu, 04 Nov 2021 12:42:54 GMT",True,"Thu, 04 Nov 2021 12:16:28 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636028504, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '22', 'color': {'name': 'blue', 'value..."
4,HQoqUKPMp8xdkDv_9iWFQ,"Thu, 04 Nov 2021 13:28:00 GMT",True,"Thu, 04 Nov 2021 12:54:03 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636031061, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '26', 'color': {'name': 'blue', 'value..."
5,dHC1CHjYJZlmeYpaEgXA9,"Thu, 04 Nov 2021 14:36:21 GMT",True,"Thu, 04 Nov 2021 14:04:53 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636035197, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '28', 'color': {'name': 'blue', 'value..."
6,U6GK38SCynnf2dwbI3VNL,"Thu, 04 Nov 2021 16:24:53 GMT",True,"Thu, 04 Nov 2021 15:46:43 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1636041884, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '26', 'color': {'name': 'blue', 'value..."
7,8vdFnoKl4lVqIPMunPpq-,"Thu, 18 Nov 2021 09:22:39 GMT",True,"Thu, 18 Nov 2021 08:47:55 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1637225895, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '23', 'color': {'name': 'blue', 'value..."
8,lt97LymL-cdjHoWWcrdTA,"Thu, 18 Nov 2021 10:34:18 GMT",True,"Thu, 18 Nov 2021 10:05:07 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1637230548, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '25', 'color': {'name': 'blue', 'value..."
9,rw56nkwwn94r78R_L94u_,"Thu, 18 Nov 2021 12:49:24 GMT",True,"Thu, 18 Nov 2021 12:17:32 GMT",2452d47b-4176-4554-abca-bc161d6b2e94,"[{'content': {}, 'end_time': 1637238313, 'star...","[{'ambient': False, 'color': [255, 255, 255], ...",500401b5-622f-48ce-8cc5-5a6d48c80391,"{'age': '24', 'color': {'name': 'blue', 'value..."


In [4]:
# Columns: 
# sessionId(The experiment session id, is the _id parameter in full results json format from platform), 
# time(the number in first row of IBI.csv file from E4 manager is initial time of the session expressed as unix timestamp in UTC)
e4_ts = path+"/input/e4_timestamps.json" 

In [5]:
# Load the empatica e4 timestamp sessions
e4_sessions = pd.read_json(e4_ts)
e4_sessions

,sessionId,empaticaId,time
0,GQOhQlQTL9rrtn0IVdMsf,1390914,1635423378
1,lvL_pTKXF0QLQiH3jsXJH,1394900,1636016893
2,OJ6xmryicQnoNVqh6f2EX,1395037,1636023307
3,HQoqUKPMp8xdkDv_9iWFQ,1395117,1636030756
4,dHC1CHjYJZlmeYpaEgXA9,1395164,1636034851
5,bZqZXJl9xdI1bgJIkefIm,1395096,1636028254
6,U6GK38SCynnf2dwbI3VNL,1395245,1636041576
7,8vdFnoKl4lVqIPMunPpq-,1404038,1637225558
8,lt97LymL-cdjHoWWcrdTA,1404078,1637230221
9,rw56nkwwn94r78R_L94u_,1404176,1637237925


In [6]:
ts = pd.DataFrame()
for i, session in sessions.iterrows():
    responses = session['responses']
    responses = pd.DataFrame.from_dict(responses)
    index = 0
    step_info = {'sessionId': session['_id'],
                 'empaticaId': str(e4_sessions[e4_sessions['sessionId'] == session['_id']].iloc[0]['empaticaId']),
                 'e4_start': e4_sessions[e4_sessions['sessionId'] == session['_id']].iloc[0]['time']}
    for index, step in responses.iterrows():
        [start, end] = [step['start_time'], step['end_time']]
        dur = end - start
        step_info.update(
            {f'step_{index}': step['step_id'],
             f'type_step_{index}': step['type'],
             f'e4_diff_step_{index}': start - step_info['e4_start'],
             f'start_step_{index}': start,
             f'end_step_{index}': end,
             f'dur_step_{index}': dur})
    ts = ts.append(step_info, ignore_index=True)
ts

,sessionId,empaticaId,e4_start,step_0,type_step_0,e4_diff_step_0,start_step_0,end_step_0,dur_step_0,step_1,...,e4_diff_step_5,start_step_5,end_step_5,dur_step_5,step_6,type_step_6,e4_diff_step_6,start_step_6,end_step_6,dur_step_6
0,GQOhQlQTL9rrtn0IVdMsf,1390914,1.635423e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,128.0,1.635424e+09,1.635424e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1254.0,1.635425e+09,1.635425e+09,299.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1554.0,1.635425e+09,1.635425e+09,340.0
1,lvL_pTKXF0QLQiH3jsXJH,1394900,1.636017e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,176.0,1.636017e+09,1.636017e+09,181.0,d3183249-d7f1-4591-8282-d927104485e1,...,1572.0,1.636018e+09,1.636019e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1873.0,1.636019e+09,1.636019e+09,321.0
2,OJ6xmryicQnoNVqh6f2EX,1395037,1.636023e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,121.0,1.636023e+09,1.636024e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1447.0,1.636025e+09,1.636025e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1747.0,1.636025e+09,1.636025e+09,170.0
3,bZqZXJl9xdI1bgJIkefIm,1395096,1.636028e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,70.0,1.636028e+09,1.636029e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,965.0,1.636029e+09,1.636030e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1265.0,1.636030e+09,1.636030e+09,255.0
4,HQoqUKPMp8xdkDv_9iWFQ,1395117,1.636031e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,125.0,1.636031e+09,1.636031e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1141.0,1.636032e+09,1.636032e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1441.0,1.636032e+09,1.636032e+09,282.0
5,dHC1CHjYJZlmeYpaEgXA9,1395164,1.636035e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,166.0,1.636035e+09,1.636035e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1204.0,1.636036e+09,1.636036e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1504.0,1.636036e+09,1.636037e+09,226.0
6,U6GK38SCynnf2dwbI3VNL,1395245,1.636042e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,127.0,1.636042e+09,1.636042e+09,181.0,d3183249-d7f1-4591-8282-d927104485e1,...,1107.0,1.636043e+09,1.636043e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1408.0,1.636043e+09,1.636043e+09,109.0
7,8vdFnoKl4lVqIPMunPpq-,1404038,1.637226e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,157.0,1.637226e+09,1.637226e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1165.0,1.637227e+09,1.637227e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1465.0,1.637227e+09,1.637227e+09,336.0
8,lt97LymL-cdjHoWWcrdTA,1404078,1.637230e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,147.0,1.637230e+09,1.637231e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1035.0,1.637231e+09,1.637232e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1335.0,1.637232e+09,1.637232e+09,102.0
9,rw56nkwwn94r78R_L94u_,1404176,1.637238e+09,15d7b80f-e18c-4668-9e3a-ea5ddb4ac82d,info,208.0,1.637238e+09,1.637238e+09,180.0,d3183249-d7f1-4591-8282-d927104485e1,...,1241.0,1.637239e+09,1.637239e+09,300.0,2452d47b-4176-4554-abca-bc161d6b2e94,question,1541.0,1.637239e+09,1.637240e+09,298.0


In [7]:
# create csv with timestamp
ts.to_csv(path+"/output/timestamp.csv", index=False)